In [ ]:
#import libraries
from osgeo import gdal, osr
import numpy as np
import glob

In [ ]:
#create function to convert array into GeoTIFF raster
def array2raster(newRasterfn, dataset, array, dtype):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    
    rows = array.shape[0]
    cols = array.shape[1]
    print ("cols and rows: ", cols, rows)
    
    originX, pixelWidth, b, originY, d, pixelHeight= dataset.GetGeoTransform()
    print ("originX, pixelWidth, b, originY, d, pixelHeight:",originX, pixelWidth, b, originY, d, pixelHeight)

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte": 
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")

    # set number of band.
    band_num = 1

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX,pixelWidth, 0, originY, 0, pixelHeight))
    originXOut, pixelWidthOut, bOut, originYOut, dOut, pixelHeightOut = outRaster.GetGeoTransform()
    print ("originXOut, pixelWidthOut, bOut, originYOut, dOut, pixelHeightOut:",originXOut, pixelWidthOut, bOut, originYOut, dOut, pixelHeightOut)
    
    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
        else:
            outband.WriteArray(array[b,:,:])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [ ]:
#define paths
path = r'C:/.../*.tif'#path of layers to extract statistics from
TMPpath="C:/.../"#output folder

#create list of files
files = glob.glob(path)
print(files)

In [ ]:
#create an empty array that has the same size as the rasters
ds1 = gdal.Open(files[0])
b1 = ds1.GetRasterBand(1)
refRaster = b1.ReadAsArray()*0

#sum all files divided by the number of files
for file in files:
    ds2 = gdal.Open(file)
    b2 = ds2.GetRasterBand(1)
    file2 = b2.ReadAsArray()#read file as a numpy array
    file2=np.divide(file2,len(files))#divide file values by the number of files (done this way to avoid loading memory with large numbers)
    refRaster=np.add(refRaster,file2)#sum values from all files
    print ("file processed: ",file)
    ds2=None
    b2=None
    file2=None   
array2raster(TMPpath+"EVI_mean.tif",gdal.Open(files[0]), refRaster, "Float32")#export file to GeoTIFF
ds1 = None
b1=None
refRaster=None       